# *Modern Deep Learning for Tabular Data*, Chapter 12

**Neural Network Interprebility**

This notebook contains the complementary code discussed in Chapter 12 of *Modern Deep Learning for Tabular Data*.

External Kaggle links to datasets used in this notebook:
- None

You can download these datasets from Kaggle, or import these notebooks into Kaggle and connect them internally.

---

## Imports

In [ ]:
# data management
import numpy as np                   # for linear algebra
import pandas as pd                  # for tabular data manipulation and processing

# machine learning
import sklearn                       # for data prep and classical ML
import tensorflow
import tensorflow as tf              # for deep learning
from tensorflow import keras         # for deep learning
import keras

# data visualization and graphics
import matplotlib.pyplot as plt      # for visualization fundamentals
import seaborn as sns                # for pretty visualizations
import cv2                           # for image manipulation

# misc
from tqdm.notebook import tqdm       # for progress bars
import math                          # for calculation
import sys                           # for system manipulation
import os                            # for file manipulation

---

## SHAP

In [ ]:
import shap
shap.initjs()

In [ ]:
x, y = shap.datasets.adult()
y = y.astype(np.int32)

from sklearn.model_selection import train_test_split as tts
X_train, X_valid, y_train, y_valid = tts(x, y, train_size = 0.8, random_state = 42)

In [ ]:
import autokeras as ak
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=20 # CHANGE THIS LATER!
)
clf.fit(X_train, y_train, epochs=50)

model = clf.export_model()

def f(X):
    return model.predict(X).flatten()
explainer = shap.KernelExplainer(f, X_valid.iloc[:100,:])

for i in [100, 101, 102]:
    shap_values = explainer.shap_values(X_valid.iloc[i], nsamples=500)
    shap.force_plot(explainer.expected_value, shap_values, X_valid.iloc[i,:])
shap_values = explainer.shap_values(X_valid.iloc[:100], nsamples=100)
shap.force_plot(explainer.expected_value, shap_values, X_valid.iloc[:100])
shap.summary_plot(shap_values, X_valid.iloc[:100])

In [ ]:
from sklearn.model_selection import train_test_split as tts

data = pd.read_csv('../input/mpempe/mouse-protein-expression.csv').drop('Unnamed: 0', axis=1)
X_train, X_valid, y_train, y_valid = tts(data.drop('class', axis=1), data['class'])
from tensorflow import keras
from keras import layers as L

model = keras.models.Sequential()
model.add(L.Input((len(X_train.columns),)))
for i in range(3):
    model.add(L.Dense(32, activation='relu'))
for i in range(2):
    model.add(L.Dense(16, activation='relu'))
model.add(L.Dense(8, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train-1, epochs=100, verbose=0)
model.evaluate(X_valid, y_valid-1)
for layer in [-1, -2, -3]:
    
    submodel = tensorflow.keras.models.Model(inputs=model.input, outputs=model.layers[layer].output)
    explainer = shap.DeepExplainer(submodel, np.array(X_train))

    values = explainer.shap_values(np.array(X_train)[:200])
    values = np.array(values)

    mean_importance = np.mean(values.reshape(-1, values.shape[-1]), axis=0)
    plt.figure(figsize=(8, 8), dpi=400)
    sns.heatmap(mean_importance.reshape((10, 8)), annot=True, xticklabels=[], yticklabels=[])
    plt.show()
layer = -1

submodel = keras.models.Model(inputs=model.input, outputs=model.layers[layer].output)
explainer = shap.GradientExplainer(submodel, np.array(X_train), local_smoothing=20)

values = explainer.shap_values(np.array(X_train)[:200])
values = np.array(values)

mean_importance = np.std(values.reshape(-1, values.shape[-1]), axis=0)
plt.figure(figsize=(8, 8), dpi=400)
sns.heatmap(mean_importance.reshape((10, 8)), annot=True, xticklabels=[], yticklabels=[])
plt.show()

---

## LIME

In [ ]:
# same setup as SHAP for comparison
from sklearn.model_selection import train_test_split as tts
X_train, X_valid, y_train, y_valid = tts(x, y, train_size = 0.7, 
                                         random_state = 42)
import autokeras as ak
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
inputs=input_node, outputs=output_node, 
overwrite=True, max_trials=20
)
clf.fit(X_train, y_train, epochs=30, batch_size=2048)
model = clf.export_model()
# custom prediction function
def pred_proba(x):
    p = model.predict(x)
    return np.concatenate([p, 1-p], axis=1)
exp = explainer.explain_instance(
    data_row=X_valid.iloc[4], 
    predict_fn=pred_proba,
    num_features=8,
    num_samples=1000,
    labels=(0,)
)

exp.show_in_notebook(show_table=True)
# change dpi for display
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

# can be changed back to default
# by setting it to 100
plot = exp.as_pyplot_figure(label=0)

---

## Activation Maximization

In [ ]:
!pip install tf_keras_vis
from keras import layers as L
from tf_keras_vis.activation_maximization import ActivationMaximization

def loss(output):
    return (output[0, 0], output[1, 1], output[2, 2], output[3, 3], output[4, 4], output[5, 5], output[6, 6], output[7, 7])

def model_modifier(model):
    model.layers[-1].activation = tensorflow.keras.activations.linear
df = pd.read_csv('../input/mpempe/mouse-protein-expression.csv').drop('Unnamed: 0', axis=1)

from sklearn.model_selection import train_test_split as tts
mpe_x = df.drop('class', axis=1)
mpe_y = df['class']
X_train, X_valid, y_train, y_valid = tts(mpe_x, mpe_y, train_size = 0.8, random_state = 42)
inp = L.Input((80,))
x = L.Dense(32, activation='relu')(inp)
for nodes in [32, 32]:
    x = L.Dense(nodes, activation='relu')(x)
for nodes in [16, 16]:
    x = L.Dense(nodes, activation='relu')(x)
x = L.Dense(8, activation='softmax')(x)
model = keras.models.Model(inputs=inp, outputs=x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train-1, epochs=50, validation_data=(X_valid, y_valid-1))
inp = L.Input((80,1,1))
reshape = L.Reshape((80,))(inp)
modelOut = model(reshape)
act = keras.models.Model(inputs=inp, outputs=modelOut)
for i in range(20):
    
    visualize_activation = ActivationMaximization(act, model_modifier)

    # Generate a random seed for each activation
    seed_input = tensorflow.random.uniform((10, 80, 1, 1), 0, 1)

    # Generate activations and convert into images
    activations = visualize_activation(loss, seed_input=seed_input, steps=256)
    images = [activation.astype(np.float32) for activation in activations]

    plt.set_cmap('gray')
    plt.figure(figsize=(9,12), dpi=400)
    for i in range(0, len(images)):
        plt.subplot(4, 3, i+1)
        visualization = images[i].reshape(8, 10)
        plt.imshow(visualization)
        plt.title(f'Target: {i}')
        plt.axis('off')
    plt.show()
    plt.close()